In [2]:
import pandas as pd

# Load the uploaded Excel file to analyze the data
file_path = 'card_sample.csv'
card_data = pd.read_csv(file_path)

# Display the first few rows of the data to understand its structure
card_data.head()

,id,card_created_at,campaign_source,device,credit_limit,risk_model_score,12mth_default
0,856843,2019-01-01,Influencer,messenger,100,0.93,0
1,857774,2019-01-01,Facebook Ads,messenger,200,0.82,0
2,859512,2019-01-02,Influencer,messenger,500,0.74,1
3,861357,2019-01-02,Influencer,ios,200,0.87,0
4,862830,2019-01-03,Facebook Ads,android,100,0.54,1


In [3]:
# Let's explore the distribution of the risk_model_score and 12mth_default to understand the default rates at various score levels.
# This will help in identifying a potential threshold for approval.

# Checking summary statistics for risk_model_score and default rates
risk_score_summary = card_data['risk_model_score'].describe()
default_rate_by_score = card_data.groupby('risk_model_score')['12mth_default'].mean()

# Calculate overall default rate as a reference
overall_default_rate = card_data['12mth_default'].mean()

# Display the summaries
risk_score_summary, default_rate_by_score.head(10), overall_default_rate

(count    50000.000000
 mean         0.620306
 std          0.187649
 min          0.300000
 25%          0.460000
 50%          0.620000
 75%          0.780000
 max          0.940000
 Name: risk_model_score, dtype: float64,
 risk_model_score
 0.30    0.843467
 0.31    0.789196
 0.32    0.733164
 0.33    0.748062
 0.34    0.835459
 0.35    0.748021
 0.36    0.970706
 0.37    0.933419
 0.38    0.633822
 0.39    0.943197
 Name: 12mth_default, dtype: float64,
 0.63708)

In [5]:
# Let's examine default rates across a range of risk model score thresholds to find a potential cutoff point
import numpy as np

# Define possible thresholds (from 0.3 to 0.9 in increments of 0.05)
thresholds = np.arange(0.3, 0.95, 0.05)
approval_rates = []
default_rates = []
expected_revenue_per_user = []

# Assume a default interest rate for simplicity, let's use 32%
interest_rate = 0.32

# Calculate metrics for each threshold
for threshold in thresholds:
    approved_users = card_data[card_data['risk_model_score'] >= threshold]
    approval_rate = len(approved_users) / len(card_data)
    default_rate = approved_users['12mth_default'].mean()
    
    # Calculate revenue per user
    avg_credit_limit = approved_users['credit_limit'].mean()
    subscription_revenue = 50
    interest_revenue = avg_credit_limit * interest_rate * (1 - default_rate)  # interest only from non-defaulters
    total_revenue = subscription_revenue + interest_revenue
    expected_revenue_per_user.append(total_revenue)
    
    # Store approval and default rates
    approval_rates.append(approval_rate)
    default_rates.append(default_rate)

# Create a DataFrame to display and analyze
strategy_df = pd.DataFrame({
    'Threshold': thresholds,
    'Approval Rate': approval_rates,
    'Default Rate': default_rates,
    'Expected Revenue per User': expected_revenue_per_user
})

In [6]:
strategy_df

,Threshold,Approval Rate,Default Rate,Expected Revenue per User
0,0.30,1.00000,0.637080,80.929378
1,0.35,0.92246,0.624244,81.993439
2,0.40,0.84650,0.604300,83.661785
3,0.45,0.76884,0.575959,86.060618
4,0.50,0.69252,0.550569,88.230170
5,0.55,0.61604,0.521265,90.625935
6,0.60,0.54096,0.494269,92.841556
7,0.65,0.46450,0.470915,94.894659
8,0.70,0.38612,0.435668,97.907857
9,0.75,0.30806,0.396416,101.357008


In [7]:
# Analyzing the threshold strategy data to find the threshold that maximizes expected revenue per user
# while keeping a reasonable approval rate.

# Setting a minimum acceptable approval rate, e.g., let's assume 50%
min_approval_rate = 0.5

# Filter for thresholds that meet the minimum approval rate
filtered_df = strategy_df[strategy_df['Approval Rate'] >= min_approval_rate]

# Find the row with the maximum expected revenue per user within the acceptable approval rate range
optimal_threshold_row = filtered_df.loc[filtered_df['Expected Revenue per User'].idxmax()]
optimal_threshold_row

Threshold                     0.600000
Approval Rate                 0.540960
Default Rate                  0.494269
Expected Revenue per User    92.841556
Name: 6, dtype: float64